In [1]:
#importing necessary libraries
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup 

In [61]:
#scraping State wise Data
url="http://swachhbharaturban.gov.in/ihhl/RPTApplicationSummary.aspx"
page = requests.get(url)
page_content = BeautifulSoup(page.content, "html.parser")
table=page_content.find('table', id='ContentPlaceHolder1_gvApplicationListState')
rows=table.findAll('tr')
rows[1].append(rows[2])
headers=rows[1].text.split('\n')
headers=[x.strip() for x in headers]
headers=[x for x in headers if x]
headers=['SNO',
 'State',
 'No. of Applications Received',
 'No. of Applications Not Verified',
 'No. of Applications Verified',
 'No. of Applications Approved',
 'No. of Applications Approved having Aadhar No.',
 'No. of Applications Rejected',
 'No. of Applications Pullback',
 'No. of Applications Closed',
 'No. of Constructed Toilet Photo Uploaded',
 'No. of Constructed Toilet Photo Approved',
 'No. of Commenced Toilet Photo ',
 'No. of Constructed Toilet Photo through Swachhalaya',
 ]
# starting 1,2 rows are for headers and 0 row was blank
full_data=[]
for i in range(3,len(rows)):
    full_data.append(rows[i].text.split('\n'))
clean_data=[]
for i in range(len(full_data)):
    space_to_empty=[x.strip() for x in full_data[i]]
    space_clean_list=[x for x in space_to_empty if x]
    clean_data.append(space_clean_list)
final_table=pd.DataFrame(clean_data)
final_table=final_table.drop_duplicates()
#column 13 just have symbol, so I am going to drop it
del final_table[13]
final_table.columns=headers
final_table.to_csv('State_wise_data')

In [62]:
#scraping state_ids and state_names to fetch district data.
timeout = 20
browser = webdriver.Firefox()
browser.get('http://swachhbharaturban.gov.in/ihhl/RPTApplicationSummary.aspx')
browser.set_page_load_timeout(timeout)
html=browser.page_source
soup=BeautifulSoup(html)
links_for_states=soup.findAll('a')
state_ids=[]
for i in range(2,len(links_for_states)-1):
    state_ids.append(links_for_states[i]['id'])
state_name=[]
for i in range(2,len(links_for_states)-1):
    state_name.append(links_for_states[i].text)
headers=['SNO',
     'District',
     'No. of Applications Received',
     'No. of Applications Not Verified',
     'No. of Applications Verified',
     'No. of Applications Approved',
     'No. of Applications Approved having Aadhar No.',
     'No. of Applications Rejected',
     'No. of Applications Pullback',
     'No. of Applications Closed',
     'No. of Constructed Toilet Photo Uploaded',
     'No. of Constructed Toilet Photo Approved',
     'No. of Commenced Toilet Photo ',
     'No. of Constructed Toilet Photo through Swachhalaya',
     ]

In [63]:
#Scraping District Wise Data
timeout = 200
browser = webdriver.Firefox()
browser.get('http://swachhbharaturban.gov.in/ihhl/RPTApplicationSummary.aspx')
browser.set_page_load_timeout(timeout)
a=pd.DataFrame()
for j in range(0,len(state_ids)):
    browser.find_element_by_id(state_ids[j]).click()
    element= WebDriverWait(browser, 10).until(lambda x: x.find_element_by_id('ContentPlaceHolder1_gvApplicationListDistrict'))
    soup2=BeautifulSoup(browser.page_source)
    soup=soup2.find('table',id='ContentPlaceHolder1_gvApplicationListDistrict')
    element2= WebDriverWait(browser, 10).until(lambda x: x.find_element_by_id('ContentPlaceHolder1_gvApplicationListDistrict'))
    rows2=soup.findAll('tr')
    full_data_district=[]
    for i in range(4,len(rows2)):
        full_data_district.append(rows2[i].text.split('\n'))
    clean_data_district=[]
    for k in range(len(full_data_district)):
        space_to_empty=[x.strip() for x in full_data_district[k]]
        space_clean_list=[x for x in space_to_empty if x]
        clean_data_district.append(space_clean_list)
    final_table_district=pd.DataFrame(clean_data_district)
    final_table_district=final_table_district.drop_duplicates()
    
    #column 13 just have symbol, so I am going to drop it
    del final_table_district[13]
    final_table_district.columns=headers
    final_table_district.insert(1,'State',state_name[j])
    a=pd.concat([a,final_table_district])
    browser.get('http://swachhbharaturban.gov.in/ihhl/RPTApplicationSummary.aspx')

browser.quit()
    
district_wise_data=a
district_wise_data=district_wise_data.dropna()
district_wise_data.to_csv('District_wise_data.csv')

In [64]:
# Scraping ULB Wise Data
timeout = 200
browser = webdriver.Firefox()
browser.get('http://swachhbharaturban.gov.in/ihhl/RPTApplicationSummary.aspx')
browser.set_page_load_timeout(timeout)
html=browser.page_source
state_html=BeautifulSoup(html)
state_tags=state_html.findAll('a')
headers=['SNO',
             'ulb',
             'No. of Applications Received',
             'No. of Applications Not Verified',
             'No. of Applications Verified',
             'No. of Applications Approved',
             'No. of Applications Approved having Aadhar No.',
             'No. of Applications Rejected',
             'No. of Applications Pullback',
             'No. of Applications Closed',
             'No. of Constructed Toilet Photo Uploaded',
             'No. of Constructed Toilet Photo Approved',
             'No. of Commenced Toilet Photo ',
             'No. of Constructed Toilet Photo through Swachhalaya',
             ]
a=pd.DataFrame()
for j in range(2,len(state_tags)-1):
    WebDriverWait(browser, 10).until(lambda x: x.find_element_by_id('ContentPlaceHolder1_gvApplicationListState'))
    state_click=browser.find_element_by_id(state_tags[j]['id'])
    state_click.click()
    state=state_tags[j].text
    element= WebDriverWait(browser, 10).until(lambda x: x.find_element_by_id('ContentPlaceHolder1_gvApplicationListDistrict'))
    soup2=BeautifulSoup(browser.page_source)
    soup=soup2.find('table',id='ContentPlaceHolder1_gvApplicationListDistrict')
    element2= WebDriverWait(browser, 10).until(lambda x: x.find_element_by_id('ContentPlaceHolder1_gvApplicationListDistrict'))
    districts=soup.findAll('a')
    for i in range(0,len(districts)):
        WebDriverWait(browser, 10).until(lambda x: x.find_element_by_id('ContentPlaceHolder1_gvApplicationListDistrict'))
        element=browser.find_element_by_id(districts[i]['id'])
        element.click()
        district=districts[i].text
        element= WebDriverWait(browser, 10).until(lambda x: x.find_element_by_id('ContentPlaceHolder1_gvApplicationListULB'))
        soup3=BeautifulSoup(browser.page_source)
        soup4=soup3.find('table',id='ContentPlaceHolder1_gvApplicationListULB')
        ulb=soup4.findAll('tr')
        if len(ulb)<4:
            b=[0]*14
            c=pd.DataFrame(b)
            c=c.transpose()
            final_table_ulb=c  
        full_data_ulb=[]
        for k in range(4,len(ulb)): 
            full_data_ulb.append(ulb[k].text.split('\n'))
        clean_data_ulb=[]
        for k in range(len(full_data_ulb)):
            space_to_empty=[x.strip() for x in full_data_ulb[k]]
            space_clean_list=[x for x in space_to_empty if x]
            clean_data_ulb.append(space_clean_list)
            final_table_ulb=pd.DataFrame(clean_data_ulb)
            final_table_ulb=final_table_ulb.drop_duplicates()
            #column 13 just have symbol, so I am going to drop it
            del final_table_ulb[13]
        
        final_table_ulb.columns=headers
        final_table_ulb.insert(1,'State',state)
        final_table_ulb.insert(2,'District',district)
        a=pd.concat([a,final_table_ulb])
        browser.find_element_by_link_text(state_tags[j].text.strip()).click()
    browser.get('http://swachhbharaturban.gov.in/ihhl/RPTApplicationSummary.aspx')
browser.quit()
ulb_wise_data=a.dropna()
ulb_wise_data.to_csv('ulb_wise_data.csv')

In [51]:
# Scraping Ward_wise_data
# In state tags, first two rows are noisy data ,so we are skipping those and counter will start from 2
# I have parsed it in six chunks,from states 1-6,6-12,12-18,18-24,24-30,30-36 as it is too much data to be parsed at once.
timeout = 200
browser = webdriver.Firefox()
browser.get('http://swachhbharaturban.gov.in/ihhl/RPTApplicationSummary.aspx')
browser.set_page_load_timeout(timeout)
browser.implicitly_wait(30)
html=browser.page_source
state_html=BeautifulSoup(html)
state_tags=state_html.findAll('a')
headers=['SNO',
             'Ward No.',
             'No. of Applications Received',
             'No. of Applications Not Verified',
             'No. of Applications Verified',
             'No. of Applications Approved',
             'No. of Applications Approved having Aadhar No.',
             'No. of Applications Rejected',
             'No. of Applications Pullback',
             'No. of Applications Closed',
             'No. of Constructed Toilet Photo Uploaded',
             'No. of Constructed Toilet Photo Approved',
             'No. of Commenced Toilet Photo ',
             'No. of Constructed Toilet Photo through Swachhalaya',
             ]
a=pd.DataFrame()
for j in range(2,len(state_tags)-1):
    WebDriverWait(browser,20).until(EC.presence_of_all_elements_located((By.ID,'ContentPlaceHolder1_gvApplicationListState')))
    state_click=browser.find_element_by_id(state_tags[j]['id'])
    state_click.click()
    state=state_tags[j].text
    element= WebDriverWait(browser, 20).until(EC.presence_of_all_elements_located((By.ID,'ContentPlaceHolder1_gvApplicationListDistrict')))
    soup2=BeautifulSoup(browser.page_source)
    soup=soup2.find('table',id='ContentPlaceHolder1_gvApplicationListDistrict')
    element2= WebDriverWait(browser, 20).until(EC.presence_of_all_elements_located((By.ID,'ContentPlaceHolder1_gvApplicationListDistrict')))
    districts=soup.findAll('a')

    for i in range(0,len(districts)):
        WebDriverWait(browser, 20).until(EC.presence_of_all_elements_located((By.ID,'ContentPlaceHolder1_gvApplicationListDistrict')))
        element=browser.find_element_by_id(districts[i]['id'])
        element.click()
        district=districts[i].text
        element= WebDriverWait(browser, 20).until(EC.presence_of_all_elements_located((By.ID,'ContentPlaceHolder1_gvApplicationListULB')))
        soup3=BeautifulSoup(browser.page_source)
        soup4=soup3.find('table',id='ContentPlaceHolder1_gvApplicationListULB')
        ulb=soup4.findAll('a')
        
        
        for k in range(0,len(ulb)):
            WebDriverWait(browser, 20).until(EC.presence_of_all_elements_located((By.ID,'ContentPlaceHolder1_gvApplicationListULB')))
            element=browser.find_element_by_id(ulb[k]['id'])
            element.click()
            ulb_name=ulb[k].text
            #for ulb having no records
            if len(ulb[k])<1:
                b=[0]*14
                c=pd.DataFrame(b)
                c=c.transpose()
                final_table_ward=c
                final_table_ward.columns=headers
                final_table_ward.insert(1,'State',state)
                final_table_ward.insert(2,'District',district)
                final_table_ward.insert(3,'ULB',ulb_name)
                a=pd.concat([a,final_table_ward])
            element=WebDriverWait(browser, 20).until(EC.presence_of_all_elements_located((By.ID,'ContentPlaceHolder1_gvApplicationListWARD')))
            soup5=BeautifulSoup(browser.page_source)
            soup6=soup5.find('table',id='ContentPlaceHolder1_gvApplicationListWARD')
            ward=soup6.findAll('tr')
            # for ward having no records
            if(len(ward))<4:
                b=[0]*14
                c=pd.DataFrame(b)
                c=c.transpose()
                final_table_ward=c 
            full_data_ward=[]
            for l in range(4,len(ward)): 
                full_data_ward.append(ward[l].text.split('\n'))
            clean_data_ward=[]
            for m in range(len(full_data_ward)):
                space_to_empty=[x.strip() for x in full_data_ward[m]]
                space_clean_list=[x for x in space_to_empty if x]
                clean_data_ward.append(space_clean_list)
                final_table_ward=pd.DataFrame(clean_data_ward)
                final_table_ward=final_table_ward.drop_duplicates()
                #column 13 just have symbol, so I am going to drop it
                del final_table_ward[13]
            final_table_ward.columns=headers
            final_table_ward.insert(1,'State',state)
            final_table_ward.insert(2,'District',district)
            final_table_ward.insert(3,'ULB',ulb_name)
            a=pd.concat([a,final_table_ward])
            browser.find_element_by_link_text(districts[i].text.strip()).click()
        browser.find_element_by_link_text(state_tags[j].text.strip()).click()
    browser.get('http://swachhbharaturban.gov.in/ihhl/RPTApplicationSummary.aspx')
browser.quit()

In [52]:
ward_wise_data=a
ward_wise_data=ward_wise_data.dropna()
ward_wise_data.to_csv('ward_wise_data.csv')

Statewise,Districtwise,ulbwise data can also be prepared by using group by on ward_wise data.Here is code for that

In [58]:
ulb_wise_data=ward_wise_data.groupby(['State','District','ULB']).sum().reset_index()
ulb_wise_data.drop(['SNO','Ward No.'],axis=1,inplace=True)
ulb_wise_data.insert(0,'SNO',1)
ulb_wise_data['SNO']=np.arange(1,len(ulb_wise_data['SNO'])+1)

In [55]:
district_wise_data=ulb_wise_data.groupby(['State','District']).sum().reset_index()
district_wise_data.drop(['SNO'],axis=1,inplace=True)
district_wise_data.insert(0,'SNO',1)
district_wise_data['SNO']=np.arange(1,len(district_wise_data['SNO'])+1)

In [56]:
state_wise_data=district_wise_data.groupby(['State']).sum().reset_index()
state_wise_data.drop(['SNO'],axis=1,inplace=True)
state_wise_data.insert(0,'SNO',1)
state_wise_data['SNO']=np.arange(1,len(state_wise_data['SNO'])+1)